# High-Level API

## Overview

The High-Level API makes it easy to rapidly:

* Prepare samples.
* Hypertune/ train queues of models. 
* Feed appropriate data/ parameters into those models.
* Evaluate model performance with metrics and plots.

It does so by wrapping and bundling together the methods of the [Low-Level API](api_low_level.html). The table below demonstrates how the high-level entities abstract abstract the low-level entities. While this abstraction eliminates many steps to enable rapid model prototyping, it comes at a cost of customization.

| High-level object | Groups together or creates the following objects                                                                  |
|:-----------------:|:-----------------------------------------------------------------------------------------------------------------:|
| `Pipeline`        | Dataset, File, Image, Tabular, Label, Featureset, Splitset, Foldset, Folds, Labelcoder, Encoderset, Featurecoders. |
| `Algorithm`       | Functions to build, train, predict, and evaluate a machine learning model.                                        |
| `Experiment`      | Algorithm, Hyperparamset, Hyperparamcombos, Queue, Job, Jobset, Result.                                           |

## Prerequisites

If you've already completed the instructions on the [Installation](installation.html) page, then let's get started.

In [4]:
import aiqc
from aiqc import datum

## 1. Pipeline

### a) Tabular Dataset

Tabular/ delimited/ flat-file `Dataset.Tabular` can be created from either Pandas DataFrames or flat files (CSV/ TSV or Parquet).

Let's grab one of AIQC's built-in datasets from the `datum` module that we imported above. This module is described in the 'Built-In Examples - Datasets' section of the documentation.

In [3]:
df = datum.to_pandas(name='iris.tsv')

The `Pipeline` process starts with raw data. A Dataset object is generated from that data and prepared for training based on the parameters the user provides to the `Pipeline.make` method. To get started, set the `dataFrame_or_filePath` equal to the dataframe we just fetched. It's the only argument that's actually required so 

Import any scikit-learn encoders that you want to use to encode labels and/ or features. Any encoders that you pass in will need to be instantiated with the attributes you want them to use.

> Reference the `Encoderset` section of the low-level API for more detail on how to include/ exclude specific `Featureset` columns by name/dtype. The `feature_encoders` argument seen below takes a list of dictionaries as input, where each dictionary contains the `**kwargs` for a `Featurecoder`.

In [4]:
from sklearn.preprocessing import OneHotEncoder, PowerTransformer

Rather than wrangling your data with many lines of data science code, just set the arguments below and AIQC takes care of the rest: stratification (including continuous dtypes), validation splits, cross-validation folds, and dtype/column specific encoders to be applied on-read. 

> Don't use `fold_count` unless your (total sample count / fold_count) still gives you an accurate representation of your sample population. You can try it with the 'iris_10x.tsv' datum.

In [5]:
splitset = aiqc.Pipeline.Tabular.make(
	dataFrame_or_filePath = df
    , dtype = None
	, label_column = 'species'
    , features_excluded = None
	, size_test = 0.24
	, size_validation = 0.12
	, fold_count = None
    , bin_count = None
    , label_encoder = OneHotEncoder(sparse=False)
    , feature_encoders = [{
        "sklearn_preprocess": PowerTransformer(method='box-cox', copy=False)
        , "dtypes": ['float64']
    }]
)


___/ featurecoder_index: 0 \_________

=> The column(s) below matched your filter(s) and were ran through a test-encoding successfully.

['sepal_length', 'sepal_width', 'petal_length', 'petal_width']

=> Done. All feature column(s) have encoder(s) associated with them.
No more Featurecoders can be added to this Encoderset.



### b) Sequence Dataset

The sequence dataset is a 3 dimensional structure intended for multi-observations per sample to enable time series analysis. 

In order to perform *supervised learning* on sequence files, you'll need both a `Dataset.Sequence` and a `Dataset.Tabular`:

* `Dataset.Sequence` is created from a homogenous 3d NumPy array.

* `Dataset.Tabular` is created as seen in the section above. It must contain 1 row per sample.

* Then a `Splitset` is constructed using:
  * The `Label` of the `Dataset.Tabular`.
  * The `Featureset` of the `Dataset.Sequence`.

In [5]:
df = datum.to_pandas('epilepsy.parquet')

In [6]:
label_df = df[['seizure']]

In [7]:
seq_ndarray3D = df.drop(columns=['seizure']).to_numpy().reshape(1000,178,1)

In [8]:
from sklearn.preprocessing import StandardScaler

In [9]:
seq_splitset = aiqc.Pipeline.Sequence.make(
    
    seq_ndarray3D = seq_ndarray3D
    , seq_feature_encoders = [{
        "sklearn_preprocess": StandardScaler()
        , "columns": '0'
    }]
    
    , tab_DF_or_path = label_df
    , tab_label_column = 'seizure'

    , size_test = 0.22
    , size_validation = 0.12
)

⏱️ Ingesting Sequences 🧬: 100%|█████████████████| 1000/1000 [00:10<00:00, 99.52it/s]



___/ featurecoder_index: 0 \_________


=> Info - System overriding user input to set `sklearn_preprocess.copy=False`.
   This saves memory when concatenating the output of many encoders.

=> The column(s) below matched your filter(s) and were ran through a test-encoding successfully.

['0']

=> Done. All feature column(s) have encoder(s) associated with them.
No more Featurecoders can be added to this Encoderset.



### c) Image Dataset

AIQC also supports image data and convolutional analysis. 

In order to perform *supervised learning* on image files, you'll need both a `Dataset.Image` and a `Dataset.Tabular`:

* `Dataset.Image` can be created from either a folder of images or a list of urls. The Pillow library is used to normalize images ingested into AIQC. Each image must be the same size (dimensions) and mode (colorscale).

* `Dataset.Tabular` is created as seen in the section above. It must contain 1 row per image.

* Then a `Splitset` is constructed using:
  * The `Label` of the `Dataset.Tabular`.
  * The `Featureset` of the `Dataset.Image`.

Again, we'll use the built-in data found in the `datum` module that we imported above.

In [6]:
df = datum.to_pandas(name='brain_tumor.csv')
image_urls = datum.get_remote_urls(manifest_name='brain_tumor.csv')

In [7]:
img_splitset = aiqc.Pipeline.Image.make(
    folderPath_or_urls = image_urls
    , pillow_save = {}
    , tabularDF_or_path = df
    , tabular_dtype = None
    , label_column = 'status'
    , label_encoder = None
    , size_test = 0.30
    , size_validation = None
    , fold_count = 4
    , bin_count = None
)

🖼️ Ingesting Images 🖼️: 100%|████████████████████████| 80/80 [00:09<00:00,  8.36it/s]


## 2. Experiment

As seen in the [Compatibility Matrix](compatibility.html), the only library supported at this point in time is `Keras` as it is the most straightforward for entry-level users. 

> You can find great examples of machine learning cookbooks on this blog: [MachineLearningMastery.com "Multi-Label Classification"](https://machinelearningmastery.com/multi-label-classification-with-deep-learning/)

In [8]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import History

### Model Functions

When we define our models, we'll do so by wrapping each phase in the following functions:

* `fn_build` contains the topology/ layers.
* `fn_train` specifies the samples and how the model should run.
* Optional and automatically determined: `fn_optimize`, `fn_predict` and `fn_lose`.

You can name the functions below whatever you want, but do not change their predetermined arugments (e.g. `features_shape`, `**hp`, `model`, etc.). These items are used behind the scenes to pass the appropriate data, parameters, and models into your training jobs.

Because these are functions, we can even play with the topology as a parameter! As demonstrated by the `if (hp['extra_layer'])` line below.

> Put a placeholder anywhere you want to try out different hyperparameters: `hp['<some_variable_name>']`. You'll get a chance to define the hyperparameters in a minute.

#### `fn_build`

In [9]:
def fn_build(features_shape, label_shape, **hp):
    model = Sequential()
    model.add(Dense(units=features_shape[0], activation='relu', kernel_initializer='he_uniform'))
    model.add(Dropout(hp['dropout_size']))
    
    if (hp['extra_layer']):
        model.add(Dense(units=hp['neuron_count'], activation='relu', kernel_initializer='he_uniform'))
        model.add(Dropout(hp['dropout_size']))
    
    model.add(Dense(units=label_shape[0], activation='softmax', name='output'))

    return model

#### `fn_train`

In [10]:
def fn_train(model, loser, optimizer, samples_train, samples_evaluate, **hp):    
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.fit(
        samples_train["features"]
        , samples_train["labels"]
        , validation_data = (
            samples_evaluate["features"]
            , samples_evaluate["labels"]
        )
        , verbose = 0
        , batch_size = 3
        , epochs = hp['epoch_count']
        , callbacks=[History()]
    )
    return model

> Reference the [low-level API documentation](api_low_level.html#Optional,-callback-to-stop-training-early.) for information on the custom 'early stopping' callbacks AIQC makes available.

### Hyperparameters

The `hyperparameters` below will be automatically fed into the functions above as `**kwargs` via the `**hp` argument we saw earlier.

For example, wherever you see `hp['neuron_count']`, it will pull from the *key:value* pair `"neuron_count": [9, 12]` seen below. Where model A will have 9 neurons and model B will have 12 neurons.

In [11]:
hyperparameters = {
	"neuron_count": [9, 12]
    , "extra_layer": [True, False]
	, "dropout_size": [0.10, 0.20]
    , "epoch_count": [50]
    , "learning_rate": [0.01]
}

Then pass these functions into the `Algorithm`.

The `library` and `analysis_type` help handle the model and its output behind the scenes. Current analysis types include: 'classification_multi', 'classification_binary', and 'regression'.

### `Experiment.make()`

Now it's time to bring together the data and logic into an `Experiment`.

In [13]:
queue = aiqc.Experiment.make(
    library = "keras"
    , analysis_type = "classification_multi"
    , fn_build = fn_build
    , fn_train = fn_train
    , splitset_id = splitset.id
    , repeat_count = 2
    , hide_test = False
    , hyperparameters = hyperparameters
    , foldset_id = None
    , fn_lose = None #automated.
    , fn_optimize = None #automated.
    , fn_predict = None #automated.
    , pick_count = None
    , pick_percent = None
)

In [14]:
queue.run_jobs()

🔮 Training Models 🔮: 100%|████████████████████████████████████████| 16/16 [01:11<00:00,  4.45s/it]


---

For more information on visualization of performance metrics, reference the [Visualization & Metrics](visualization.html) documentation.